In [22]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

In [23]:
# Definir la ruta de la base de datos
database_path = '../../database/105_classes_pins_dataset/'

In [24]:
# Preprocesamiento de datos con ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Especificar el split de entrenamiento y validación

In [ ]:
# Generadores de datos para entrenamiento y validación
train_generator = train_datagen.flow_from_directory(
    database_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Indicar que es el subset de entrenamiento

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    database_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Indicar que es el subset de validación

Found 3463 images belonging to 105 classes.


In [ ]:
# Inicializar el modelo
model = Sequential()

In [ ]:
# Agregar capas convolucionales y de pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

In [ ]:
# Agregar capas fully connected
model.add(Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))  # Número de clases


In [ ]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=10,
          validation_data=validation_generator,
          validation_steps=len(validation_generator))

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

# Definir la ruta de la base de datos
database_path = '../../database/105_classes_pins_dataset/'

# Preprocesamiento de datos con ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Especificar el split de entrenamiento y validación

# Generadores de datos para entrenamiento y validación
train_generator = train_datagen.flow_from_directory(
    database_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Indicar que es el subset de entrenamiento

validation_generator = train_datagen.flow_from_directory(
    database_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Indicar que es el subset de validación

num_classes = len(train_generator.class_indices)  # Obtener el número de clases

# Construir el modelo
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
# Añadir más capas o ajustar el modelo según sea necesario

# Capa de salida con el número correcto de clases y activación softmax
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo usando la GPU si está disponible
with tf.device('/GPU:1'):
    model.fit(train_generator,
              steps_per_epoch=len(train_generator),
              epochs=10,
              validation_data=validation_generator,
              validation_steps=len(validation_generator))


In [1]:
import tensorflow as tf

# Verificar si hay GPU disponible
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(f"Detalles de la GPU utilizada:\n{gpu}")
        print("Memoria disponible:", tf.config.experimental.get_memory_info(gpu))
else:
    print("No se ha detectado una GPU.")


No se ha detectado una GPU.


In [5]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preprocesamiento de datos con ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '../../database/105_classes_pins_dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    '../../database/105_classes_pins_dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Obtener imágenes y etiquetas
X_train, y_train = next(train_generator)
X_val, y_val = next(validation_generator)

# Aplanar las imágenes para PCA
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_val_flatten = X_val.reshape(X_val.shape[0], -1)

# Normalizar los datos (opcional)
X_train_flatten = X_train_flatten / 255.0
X_val_flatten = X_val_flatten / 255.0

# Aplicar PCA
pca = PCA(n_components=100)  # Número de componentes deseados
pca.fit(X_train_flatten)

# Transformar los datos de entrenamiento y validación con PCA
X_train_pca = pca.transform(X_train_flatten)
X_val_pca = pca.transform(X_val_flatten)

# Definir la arquitectura de la red neuronal
model = Sequential([
    Dense(128, activation='relu', input_shape=(100,)),  # 100 es el número de componentes de PCA
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # num_classes es el número de clases de actores que tienes
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo con las características transformadas por PCA
model.fit(X_train_pca, y_train, epochs=10, validation_data=(X_val_pca, y_val))


Found 17534 images belonging to 105 classes.


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'ruta/datos_validacion'